# Live Inference on WikiMedia Data

This notebook illustrates a simple ML pipeline that leverages `csp` for feature generation and real-time (i.e. live) inference. 

The data we are going to leverage is the [MediaWiki Recent Changes feed](https://www.mediawiki.org/wiki/Manual:RCFeed) stream, which emits events related to recent changes across Wikimedia sites. The stream can be accessed through the [EventStreams web service](https://wikitech.wikimedia.org/wiki/Event_Platform/EventStreams_HTTP_Service). 
Our objective will be to predict whether a change is being made by a bot account, without leveraging the the `bot` indicator on the events or the account name (as bot accounts typically are good citizens and have the word "bot" in their name). 

The high-level outline of this notebook is as follows
 * **Historical Data Collection**: Pull historical data from the stream, which will give us the ground truth (the "bot") flag and some metadata for generating features
 * **Feature Generation**: Write some simple feature generation logic using `csp`, and run it on the historical data to generate a feature set
 * **Model Training**: Train a classification model on the data with `sklearn`
 * **Real Time Data Adapter**: Write a real time data adapter for `csp` (see the [csp wikimedia example](https://github.com/Point72/csp/blob/main/examples/07_end_to_end/wikimedia.ipynb) for a more in-depth explanation)
 * **Real Time Inference and Monitoring**: Write simple real-time inference and monitoring nodes with `csp`
 * **Run Real Time Inference!**

To run this notebook, we need to install a few extra dependencies, listed below:
```
pip install sseclient
pip install perspective-python[jupyter]
pip install scikit-learn
```

In [ ]:
import json
import threading
from datetime import date, datetime, timedelta
from typing import Any, Dict, Optional

import numpy as np
from perspective.widget import PerspectiveWidget
from sseclient import SSEClient as EventSource

import csp
from csp import Outputs, ts
from csp.typing import NumpyNDArray
from csp.utils.datetime import utc_now

np.set_printoptions(edgeitems=5, linewidth=120)

## Historical Data Collection

In this stage, we pull a fixed number of events from the recent change feed dating back several days. This historical data will be used to get the groud-truth (the labels for our model) as well as the raw data we will use for feature generation. 

In [2]:
%%time  
# Takes about a minute

# We pull the most recent data because older data is purged from the feed
since = date.today() - timedelta(days=7)
URL = f"https://stream.wikimedia.org/v2/stream/recentchange?since={since}"
N = 100_000  # Maximum number of events to pull
data = []
for idx, item in enumerate(EventSource(URL)):
    if item.event == "message":
        try:
            change = json.loads(item.data)
        except ValueError:
            continue
        # discard canary events
        if change["meta"]["domain"] == "canary" or change["type"] not in ("new", "edit"):
            continue
        data.append(change)
        if idx >= N:
            break
data = sorted(data, key=lambda d: d["timestamp"])
print(f"{len(data)} records")


55776 records
CPU times: user 13 s, sys: 1.86 s, total: 14.8 s
Wall time: 27.5 s


Below is what a sample event looks like. Take note of the `bot` key, which will tell us whether a given event was created by a bot.

In [3]:
data[0]

{'$schema': '/mediawiki/recentchange/1.0.0',
 'meta': {'uri': 'https://en.wikipedia.org/wiki/Wikipedia:Equity_lists/Nationality/Malawi',
  'request_id': 'e068d678-ac33-41a4-a049-f365a287576d',
  'id': 'd57753bd-6b42-4f09-8a19-96f5f3d8c558',
  'dt': '2024-11-25T10:12:24Z',
  'domain': 'en.wikipedia.org',
  'stream': 'mediawiki.recentchange',
  'topic': 'codfw.mediawiki.recentchange',
  'partition': 0,
  'offset': 1274922687},
 'id': 1844881588,
 'type': 'edit',
 'namespace': 4,
 'title': 'Wikipedia:Equity lists/Nationality/Malawi',
 'title_url': 'https://en.wikipedia.org/wiki/Wikipedia:Equity_lists/Nationality/Malawi',
 'comment': 'Wikidata list updated [V2]',
 'timestamp': 1732529544,
 'user': 'ListeriaBot',
 'bot': True,
 'notify_url': 'https://en.wikipedia.org/w/index.php?diff=1259472355&oldid=1258389456',
 'minor': False,
 'length': {'old': 1067640, 'new': 1118836},
 'revision': {'old': 1258389456, 'new': 1259472355},
 'server_url': 'https://en.wikipedia.org',
 'server_name': 'en.wi

## Feature Generation

The next step of the pipeline is to generate features. 

Here we use a simple python function to generate a dictionary of features from the event dictionary, including a one-hot encoding of the `server_name` field.

In [4]:
def generate_features_from_event(event: Dict[str, Any], server_map: Dict[str, str]) -> Dict[str, Any]:
    features = {}
    # Feature engineering - can do in pandas, but also wrap in csp
    features["length.new"] = event["length"]["new"]
    features["length.delta"] = event["length"]["new"] - event["length"].get("old", 0)
    features["comment.len"] = len(event["comment"])
    # See https://www.mediawiki.org/wiki/Manual:Namespace
    features["namespace.main"] = int(event["namespace"] == 0)
    # features["namespace.talk"] = int(event["namespace"]) % 2
    # features["timestamp.hour"] = datetime.utcfromtimestamp(event["timestamp"]).hour
    # features["minor"] = int(event["minor"])

    # One hot encoding of specific recognized domains (default is zero for all)
    # features["server.main"] = int(event["server_name"] in server_map)
    for k, v in server_map.items():
        features[f"server_{v}"] = int(event["server_name"] == k)
    return features

We run the feature generation function on a sample event to illustrate the output:

In [5]:
server_map = {"www.wikidata.org": "wikidata", "commons.wikimedia.org": "wikimedia", "en.wikipedia.org": "wikipedia"}
generate_features_from_event(data[0], server_map)

{'length.new': 1118836,
 'length.delta': 51196,
 'comment.len': 26,
 'namespace.main': 0,
 'server_wikidata': 0,
 'server_wikimedia': 0,
 'server_wikipedia': 1}

Now we wrap the `generate_features_from_event` python function in a `csp` node that also converts the feature dictionary to a numpy array (which we will pass to `sklearn`):

In [6]:
@csp.node
def generate_features(event: ts[dict], server_map: dict) -> ts[NumpyNDArray[float]]:
    features = generate_features_from_event(event, server_map)
    return np.array(list(features.values()))

In this simple example, the feature generation is represented by a single node, and the features are a function of the current event only, but with `csp`, it would be possible to build a feature graph consisting of multiple nodes, and where the features depended on state built from past events (i.e. recent events counts, etc).

With the tools above in place, we can now generate a set of historical features with csp. We pass the historical events to csp using `csp.curve` - associating each event with a timestamp for the csp engine. 
By calling `csp.run` on the `generate_features` node, we produce two arrays - one of the output timestamps and one of the feature arrays. To get a 2-d array of features, we stack the output features. 

In [7]:
hist_input = csp.curve(dict, [(datetime.utcfromtimestamp(event["timestamp"]), event) for event in data])
times, features = csp.run(generate_features, hist_input, server_map, starttime=datetime(2020, 1, 1), output_numpy=True)[
    0
]
X = np.vstack(features)
print(X.shape)
X

(55776, 7)


array([[1118836,   51196,      26,       0,       0,       0,       1],
       [ 463624,      14,      27,       0,       0,       0,       0],
       [ 306317,     159,      94,       0,       0,       0,       1],
       [  64988,       0,      19,       1,       0,       0,       1],
       [ 207533,    -159,     135,       1,       0,       0,       1],
       ...,
       [  12902,       3,     127,       1,       1,       0,       0],
       [  20075,       3,     123,       1,       1,       0,       0],
       [  20078,       3,     126,       1,       1,       0,       0],
       [  85265,     288,     132,       1,       1,       0,       0],
       [  14524,      -1,      58,       1,       1,       0,       0]])

To generate the labels, we simply pull the `bot` field from each event and put the results in an array.

In [8]:
y = np.array([int(event["bot"]) for event in data])

The reason that we don't use this strategy for the feature generation is because we want to be sure that we are using the same feature generation logic in real time (where we are running a csp graph) as we do in simulation. 

As feature graphs get more complex and as they depend on stateful features or multiple asynchronous feeds, this becomes more important than in this simple example in which features are just a function of each event coming from a single source.

## Model Training

To train a model on the generated features, we use the `RandomForestClassifier` from `sklearn` with some parameters chosen from their [Classifier Comparison](https://scikit-learn.org/1.5/auto_examples/classification/plot_classifier_comparison.html). The model appears to do well enough on the historical data.

In [9]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=False)

clf0 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42)
clf = make_pipeline(StandardScaler(), clf0)
clf.fit(X_train, y_train)

# Compute the score
score = clf.score(X_test, y_test)
print("Accuracy: ", score)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Print classification report
print("Classification Report")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

Accuracy:  0.9386401326699834
Classification Report
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      9506
           1       0.98      0.91      0.94     12805

    accuracy                           0.94     22311
   macro avg       0.94      0.94      0.94     22311
weighted avg       0.94      0.94      0.94     22311

Confusion Matrix
[[ 9250   256]
 [ 1113 11692]]
CPU times: user 348 ms, sys: 47.6 ms, total: 396 ms
Wall time: 612 ms


Now we have a trained model, `clf` that we will plug into live inference later on...

## Real Time Data Adapter

Below is a real time `csp` adapter to stream the data. For a more in-depth overview of how this adapter works, refer to the [csp wikimedia example](https://github.com/Point72/csp/blob/main/examples/07_end_to_end/wikimedia.ipynb).
The main difference in the adapter below is that we publish each event as a `dict` (instead of a `csp.Struct`).

In [10]:
from csp.impl.pushadapter import PushInputAdapter
from csp.impl.wiring import py_push_adapter_def


# Define the runtime implementation of our adapter
class FetchWikiDataAdapter(PushInputAdapter):
    def __init__(self, url: str):
        self._thread = None
        self._running = False
        self._url = url

    def start(self, starttime, endtime):
        print("FetchWikiDataAdapter::start")
        self.endtime = endtime
        self.starttime = starttime
        self._source = EventSource(self._url)
        self._running = True
        self._thread = threading.Thread(target=self._run)
        self._thread.start()

    def stop(self):
        print("FetchWikiDataAdapter::stop")
        if self._running:
            self._running = False
            self._thread.join()
            self._source.resp.close()

    def _run(self):
        for item in self._source:
            if not self._running:
                break
            if item.event == "message":
                try:
                    change = json.loads(item.data)
                except ValueError:
                    pass
                else:
                    # discard canary events
                    if change["meta"]["domain"] == "canary" or change["type"] not in ("new", "edit"):
                        continue

                    self.push_tick(change)

        return None


# Create the graph-time representation of our adapter
FetchWikiData = py_push_adapter_def("FetchWikiData", FetchWikiDataAdapter, csp.ts[dict], url=str)

## Real Time Inference and Monitoring

The inference node (which can be used either in real-time or historically) simply takes a model and a stream of feature arrays, and calls predict on the arrays. It outputs the result on the `predictions` output edge in the graph. If there are any exceptions, it outputs the message on the `errors` edge.

In [11]:
@csp.node
def inference_node(model: object, features: ts[NumpyNDArray[float]]) -> Outputs(predictions=ts[bool], errors=ts[str]):
    try:
        pred = model.predict(features.reshape(1, -1))
        csp.output(predictions=bool(pred[0]))
    except Exception as e:
        csp.output(errors=str(e))

Next, we write a node that will update a Perspective Widget so that we can track performance in real time. Note that we buffer the events to limit the rate of UI updates

In [12]:
@csp.node
def update_widget(
    event: ts[dict], prediction: ts[bool], widget: PerspectiveWidget, throttle: timedelta = timedelta(seconds=0.5)
):
    # Updates the perspective widget with batched updates for scalability
    with csp.alarms():
        alarm = csp.alarm(bool)

    with csp.state():
        s_buffer = []

    with csp.start():
        csp.schedule_alarm(alarm, throttle, True)

    if csp.ticked(event, prediction):
        s_buffer.append(
            {
                "prediction": prediction,
                "bot": event["bot"],
                "timestamp": event["timestamp"] * 1e3,
                "user": event.get("user"),
                "title": event.get("title"),
                "domain": event["meta"]["domain"],
            }
        )

    if csp.ticked(alarm):
        if len(s_buffer) > 0:
            widget.update(s_buffer)
            s_buffer = []

        csp.schedule_alarm(alarm, throttle, True)

Lastly, we put it all together in the inference graph. This graph takes a model and a widget, and then 
  * generates features
  * performs inference
  * publishes the outputs.

In [13]:
@csp.graph
def inference_graph(model: object, widget: Optional[PerspectiveWidget]):
    server_map = {"www.wikidata.org": "wikidata", "commons.wikimedia.org": "wikimedia", "en.wikipedia.org": "wikipedia"}
    raw_input = FetchWikiData(url="https://stream.wikimedia.org/v2/stream/recentchange")
    features = generate_features(raw_input, server_map)
    outputs = inference_node(model, features)
    if widget:
        update_widget(raw_input, outputs.predictions, widget)
    else:  # In case you want to run without perspective, pass widget = None
        csp.print("Predictions", outputs.predictions)
        csp.print("Target", csp.apply(raw_input, lambda d: d["bot"], bool))
    csp.print("Errors", outputs.errors)

## Run Real-Time Inference!

First we create the perspective Widget to visualize the streaming data later. The widget starts empty, but will start displaying data when `csp` runs the graph. 

(Note that if you are running this example locally on JupyterLab, you may need to restart the Jupyter server after installing the Perspective library in order to visualize the widget.)

We group the results by the actual `bot` flag (true/false) and then by the predicted value (true/false) in order to form the confusion matrix. We also show the latest timestamp for each record so we can convince ourselves it's running on live events.

In [14]:
schema = {
    "prediction": "boolean",
    "bot": "boolean",
    "user": "string",
    "timestamp": "datetime",
    "title": "string",
    "domain": "string",
}
# widget = PerspectiveWidget(schema, group_by=["bot", "prediction"], columns=["prediction", "timestamp"], aggregates={"prediction": "count", "timestamp": "last"}, binding_mode="client-server")
widget = PerspectiveWidget(
    schema,
    plugin="X Bar",
    group_by=["bot", "prediction"],
    columns=["prediction"],
    aggregates={"prediction": "count", "timestamp": "last"},
    binding_mode="client-server",
)
widget

PerspectiveWidget(aggregates={'prediction': 'count', 'timestamp': 'last'}, binding_mode='client-server', column…

Next we run the graph against the previously trained model and the widget. Data should start appearing in the widget above.

In [ ]:
start = utc_now()
csp.run(inference_graph, clf, widget=widget, starttime=start, endtime=start + timedelta(seconds=30), realtime=True)
print("Done.")

FetchWikiDataAdapter::start
FetchWikiDataAdapter::stop
Done.


## Raw Data Exploration

We can also use perspective to explore the raw historical data:

In [16]:
import pandas as pd

PerspectiveWidget(pd.json_normalize(data))

PerspectiveWidget(binding_mode='server', columns=['index', '$schema', 'id', 'type', 'namespace', 'title', 'tit…